In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mne 
import scipy.signal as sp_signal
from sktime.libs.vmdpy import VMD
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import periodogram
from sklearn.preprocessing import MinMaxScaler
from pymultifracs.viz import plot_psd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "scripts")))
from simulation import generate_synthetic_neural_signal
from decomposition import run_all_decompositions
from plotting import plot_signal_and_modes
from features import compute_features_from_modes_and_save
from mfa_utils import compute_mfa, plot_mfa, plot_psd, analyze_signal_and_modes
from evaluation import summarize_decomposition_results
from pymultifracs import mfa, wavelet_analysis
import torch



# Go up two levels from notebooks/vmd/ to reach src/mvmd/
mvmd_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'mvmd'))
sys.path.append(mvmd_path)

# Now import mvmd_optimized
from mvmd_original import mvmd 
from mvmd_optimized import mvmd as mvmd_opt

## 1. Signal extraction

In [3]:
#raw = mne.io.read_raw_fif("../data/sub-01_ses-01_task-rest_proc-filt_raw.fif")
raw = mne.io.read_raw_fif("../data/real/sub-01_ses-01_task-rest_proc-filt_raw.fif", preload=True)

Opening raw data file ../data/real/sub-01_ses-01_task-rest_proc-filt_raw.fif...
    Range : 34500 ... 116999 =    138.000 ...   467.996 secs
Ready.
Reading 0 ... 82499  =      0.000 ...   329.996 secs...


## 3. Signal decomposition

### Set Parameters:

In [4]:
# Load MEG data
raw_meg = raw.copy().pick('meg')
data_matrix = raw_meg.get_data()  
data_matrix_small = data_matrix[:10, :]  # select first 10 channels


# Parameters (from your code)
fs = 250  # Sampling frequency
duration = 300.0  # Duration to plot (seconds)
max_points = 1000
methods = ["MVMD"]


best_vmd_params = {
    'meg': {'vmd_alpha': 1, 'vmd_tau': 0, 'vmd_DC': 1, 'vmd_init': 0, 'vmd_tol': 1e-1, 'Nmodes': 5},
}

output_base = "../results/real/"

n_channels_to_process = 302  # First 10 magnetometers

In [5]:
# Load MEG data
raw_meg = raw.copy().pick('meg')
#data_matrix = raw_meg.get_data()  
data_matrix_small = data_matrix[:10, :]  # select first 10 channels


In [4]:
from mvmd import mvmd

# Example signal
signal = np.random.rand(3, 100)  # 3 channels, 100 samples

# Decompose the signal
modes, modes_hat, omega = mvmd(signal, num_modes=3, alpha=2000, tolerance=1e-3, sampling_rate=1)

ImportError: cannot import name 'mvmd' from 'mvmd' (unknown location)

In [ ]:
signal_tensor = torch.tensor(data_matrix, dtype=torch.float32)  # shape: (n_channels, n_samples)

: 

In [ ]:
u, u_hat, omega = mvmd_opt(signal_tensor, alpha=1, tau=0, K=3, DC=0, init=1, tol=1e-6, max_N=5)


In [ ]:
print("u shape:", u.shape)
print("u_hat shape:", u_hat.shape)
print("omega shape:", omega.shape)


u shape: torch.Size([3, 82500, 306])
u_hat shape: torch.Size([82500, 3, 306])
omega shape: torch.Size([1, 3])


In [ ]:
print("u min:", u.min().item())
print("u max:", u.max().item())
print("u mean:", u.mean().item())
print("u std:", u.std().item())


u min: 0.0
u max: 0.0
u mean: 0.0
u std: 0.0
